In [2]:
import json
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [5]:
DATA_PATH="data_for_genre_classification.json"

#creating a function to load in the data
def load_data(DATA_PATH):
    with open(DATA_PATH,"r")as fp:#filepath
        data=json.load(fp)
    
    X=np.array(data["mfcc"])#features
    Y=np.array(data["labels"])#labels to make predictions on
    return X,Y

def prepare_dataset(test_size,validation_size):
    #loading data
    X,Y=load_data(DATA_PATH)
    #splitting dataset into train test and validation
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=test_size)
    X_train,X_validation,Y_train,Y_validation=train_test_split(X,Y,test_size=validation_size)
    #add an axis to input sets
    X_train = X_train[...,np.newaxis]
    X_validation=X_validation[...,np.newaxis]
    X_test=X_test[...,np.newaxis]
    
    return X_train,X_validation,X_test,Y_train,Y_validation,Y_test

def build_model(input_shape):
    #building cnn model to call in the main function
    model = keras.Sequential()
    # 1st  layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    # 2nd  layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    # 3rd  layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
   # output layer
    model.add(keras.layers.Dense(10, activation='softmax')) #to make predictions

    return model

def predict(model,X,Y):
    X=X[np.newaxis,...]#(array with 4 elements)
    prediction=model.predict(X)
    predicted_index=np.argmax(prediction,axis=1)
    print("Target:{},Predicted index :{}".format(Y,predicted_index))
    
    
if __name__=="__main__":
    #get train,validation,test splits
    X_train,X_validation,X_test,Y_train,Y_validation,Y_test=prepare_dataset(0.25,0.20)
    input_shape=(X_train.shape[1],X_train.shape[2],1)
    model = build_model(input_shape)
    #complie
    optimiser = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimiser,loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
    
    model.summary()
    test_error, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)
    print('\nTest accuracy: {}',test_accuracy)
    
    #picking samples to predict
    X_predict = X_test[100]
    Y_predict = Y_test[100]
    
    #use the samples to feed in the model
    predict(model,X_predict,Y_predict)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 128, 11, 32)       320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 6, 32)        0         
 2D)                                                             
                                                                 
 batch_normalization_6 (Batc  (None, 64, 6, 32)        128       
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 62, 4, 32)         9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 31, 2, 32)        0         
 2D)                                                             
                                                      